In [28]:
%autosave 300 

Autosaving every 300 seconds


In [6]:
from IPython.display import Image

In [19]:
import nltk

# Learning to Classify Text

## 학습목표

1. 언어 데이터에서 분류를 위해 특징적인 피처들을 어떻게 알아낼 수 있는가? <br>
2. 자동 언어 처리를 위해 필요한 언어 모델을 어떻게 만드는가? <br>
3. 이러한 모델들을 통하여 우리가 언어에 대해 배울 수 있는 것은 무엇인가? 

### 1. 지도 분류 (Supervised Classification)

분류란 주어진 인풋에 대하여 알맞은 'class label'을 선택하는 것 <br>
예: <br>
<ul>
<li> 이메일이 스팸인지 아닌지 </li>
<li> 뉴스 기사의 주제가 무엇인지 </li>
<li> 단어의 뜻이 무엇으로 쓰였는지 (뜻이 여러 개인 경우) </li>
</ul>
<br>
분류기는 학습 코퍼스가 각각의 인풋에 대해 정확한 레이블을 가지고 있을 때 '지도'되었다고 한다.

<img src = "images/supervised.png">

## 1.1 Gender Identification

In [10]:
# 분류기를 만들 때 중요한 것은 어떤 인풋 피처들이 관련성이 높은지, 그리고 그 피처들을 어떻게 인코딩할것인지이다.
# 예를 들면, a, e, i로 끝나는 이름들은 여성일 확률이 높고, k, o, r, s 등은 남성일 확률이 높다.
def gender_features(word):
    return {'last_letter': word[-1]} # feature set: feature name: 'last_letter'
gender_features('Shrek')

{'last_letter': 'k'}

In [13]:
from nltk.corpus import names
labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
    [(name, 'female') for name in names.words('female.txt')])
import random
random.shuffle(labeled_names)

In [14]:
names.words('male.txt')

['Aamir',
 'Aaron',
 'Abbey',
 'Abbie',
 'Abbot',
 'Abbott',
 'Abby',
 'Abdel',
 'Abdul',
 'Abdulkarim',
 'Abdullah',
 'Abe',
 'Abel',
 'Abelard',
 'Abner',
 'Abraham',
 'Abram',
 'Ace',
 'Adair',
 'Adam',
 'Adams',
 'Addie',
 'Adger',
 'Aditya',
 'Adlai',
 'Adnan',
 'Adolf',
 'Adolfo',
 'Adolph',
 'Adolphe',
 'Adolpho',
 'Adolphus',
 'Adrian',
 'Adrick',
 'Adrien',
 'Agamemnon',
 'Aguinaldo',
 'Aguste',
 'Agustin',
 'Aharon',
 'Ahmad',
 'Ahmed',
 'Ahmet',
 'Ajai',
 'Ajay',
 'Al',
 'Alaa',
 'Alain',
 'Alan',
 'Alasdair',
 'Alastair',
 'Albatros',
 'Albert',
 'Alberto',
 'Albrecht',
 'Alden',
 'Aldis',
 'Aldo',
 'Aldric',
 'Aldrich',
 'Aldus',
 'Aldwin',
 'Alec',
 'Aleck',
 'Alejandro',
 'Aleks',
 'Aleksandrs',
 'Alessandro',
 'Alex',
 'Alexander',
 'Alexei',
 'Alexis',
 'Alf',
 'Alfie',
 'Alfonse',
 'Alfonso',
 'Alfonzo',
 'Alford',
 'Alfred',
 'Alfredo',
 'Algernon',
 'Ali',
 'Alic',
 'Alister',
 'Alix',
 'Allah',
 'Allan',
 'Allen',
 'Alley',
 'Allie',
 'Allin',
 'Allyn',
 'Alonso',


In [16]:
# 피처셋 (각 이름의 마지막 글자) 만들기
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]

In [17]:
# 데이터를 나누자
train_set, test_set = featuresets[500:], featuresets[:500]

In [20]:
# 분류기를 만들자
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [21]:
# 예를 들어보자
classifier.classify(gender_features('Neo'))

'male'

In [22]:
classifier.classify(gender_features('Trinity'))

'female'

In [24]:
# 우리가 만든 분류기 평가
print(nltk.classify.accuracy(classifier, test_set))

0.754


In [25]:
# 무슨 피처가 제일 중요했나?
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     38.6 : 1.0
             last_letter = 'k'              male : female =     33.0 : 1.0
             last_letter = 'v'              male : female =     18.5 : 1.0
             last_letter = 'f'              male : female =     15.9 : 1.0
             last_letter = 'p'              male : female =     11.1 : 1.0


In [27]:
# 메모리를 너무 많이 차지하니, 리스트로 저장하되 메모리는 차지하지 않도록 하자
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

## 1.2 Choosing the Right Features

적절한 피처를 찾는 일은 매우 중요하다 (당연) 보통 "kitchen sink" 방법을 사용한다 -- 생각해낼 수 있는 모든 피처를 찾은 다음 어떤 피처가 도움이 되는지 좁혀 나가는 방법

In [31]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [32]:
gender_features2('John')

{'count(a)': 0,
 'count(b)': 0,
 'count(c)': 0,
 'count(d)': 0,
 'count(e)': 0,
 'count(f)': 0,
 'count(g)': 0,
 'count(h)': 1,
 'count(i)': 0,
 'count(j)': 1,
 'count(k)': 0,
 'count(l)': 0,
 'count(m)': 0,
 'count(n)': 1,
 'count(o)': 1,
 'count(p)': 0,
 'count(q)': 0,
 'count(r)': 0,
 'count(s)': 0,
 'count(t)': 0,
 'count(u)': 0,
 'count(v)': 0,
 'count(w)': 0,
 'count(x)': 0,
 'count(y)': 0,
 'count(z)': 0,
 'first_letter': 'j',
 'has(a)': False,
 'has(b)': False,
 'has(c)': False,
 'has(d)': False,
 'has(e)': False,
 'has(f)': False,
 'has(g)': False,
 'has(h)': True,
 'has(i)': False,
 'has(j)': True,
 'has(k)': False,
 'has(l)': False,
 'has(m)': False,
 'has(n)': True,
 'has(o)': True,
 'has(p)': False,
 'has(q)': False,
 'has(r)': False,
 'has(s)': False,
 'has(t)': False,
 'has(u)': False,
 'has(v)': False,
 'has(w)': False,
 'has(x)': False,
 'has(y)': False,
 'has(z)': False,
 'last_letter': 'n'}

In [33]:
# overfitting을 주의해야 한다 (특히 학습 데이터가 적을 때)
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]

In [34]:
train_set, test_set = featuresets[500:], featuresets[:500]

In [35]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [36]:
print(nltk.classify.accuracy(classifier, test_set))

0.782


In [37]:
# error analysis를 통해 피처를 줄여 갈 수 있다
# development set = training set + dev-test set
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

<img src = "images/test_train.png">

In [39]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]

In [40]:
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]

In [41]:
test_set = [(gender_features(n), gender) for (n, gender) in test_names]

In [42]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [43]:
print(nltk.classify.accuracy(classifier, devtest_set))

0.776


In [44]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))

In [45]:
for (tag, guess, name) in sorted(errors):
    print('correct={:<8} guess={:8} name={:30}'.format(tag, guess, name))

correct=female   guess=male     name=Agnes                         
correct=female   guess=male     name=Alisun                        
correct=female   guess=male     name=Allyson                       
correct=female   guess=male     name=Ann                           
correct=female   guess=male     name=Arleen                        
correct=female   guess=male     name=Avrit                         
correct=female   guess=male     name=Bab                           
correct=female   guess=male     name=Beitris                       
correct=female   guess=male     name=Bev                           
correct=female   guess=male     name=Brigid                        
correct=female   guess=male     name=Candis                        
correct=female   guess=male     name=Carlynn                       
correct=female   guess=male     name=Carolan                       
correct=female   guess=male     name=Caryn                         
correct=female   guess=male     name=Charin     

In [46]:
# 성별을 맞추는 데 한 글자 이상의 글자들이 필요할 수 있다는 것을 확인하였다
# 이제 두 글자 접미사를 사용하여 성별을 구분해 보자
def gender_features(word):
    return {'suffix1': word[-1:],
           'suffix2': word[-2:]}

In [47]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]

In [48]:
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]

In [49]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [50]:
print(nltk.classify.accuracy(classifier, devtest_set)) # 정확도가 조금 증가함 

0.781


## 1.3 Document Classification

In [ ]:

classifier 